# Run 'nibetaseries' package on all subjects

In [1]:
import os  # interact with the filesystem
from subprocess import Popen, PIPE, STDOUT  # enable calling commandline
import matplotlib.pyplot as plt  # manipulate figures
import seaborn as sns  # display results
import pandas as pd   # manipulate tabular 
import warnings
import time
warnings.filterwarnings("ignore", category=DeprecationWarning) 
warnings.filterwarnings("ignore", category=UserWarning) 
warnings.filterwarnings("ignore", category=ResourceWarning) 

## I. Data directory set up

## II. TSV files set up   
Origianl Betaseries Events file: `sub-001_run01_task-pst_events.tsv`  
BIDS events file: `sub-001_task-prob_run-1_events.tsv`

#### Go to setup_tsvs.py to see example code 

In [ ]:
data_dir = '/projects/niblab/bids_projects/Experiments/Bevel/testing_beta' #RENCI PATH
#data_dir = '/Users/nikkibytes/Documents/niblunc/testing_beta' #LOCAL PATH
print('Our working directory: {}'.format(data_dir))